# 관광객 추정 버스 이용자 추출 알고리즘
* input: 버스 이용 데이터
* ouput: [user_id, first_usage_date, last_usage_date, tourist_flag]

In [18]:
import os
import datetime
import pandas as pd
from tqdm import tqdm
from pyarrow import csv

# 사용자 정의
import bus

In [19]:
# 관광객 연속 체류 기간 (default: 15)
day = 15

geton_station_name = "geton_stataion_name"

### 1. Def

#### 1) 데이터 전처리
* 날짜 데이터 int -> datetime 변환

In [20]:
def data_preprocessing(usage_df):
    # datetime64로 형 변환 # M[base_date] = pd.to_datetime(M[base_date], format='%Y%m%d')
    datetime_cols = ["geton_datetime", "getoff_datetime"]    
    for col in tqdm(datetime_cols):
        usage_df[col] = pd.to_datetime(usage_df[col], format='%Y%m%d%H%M%S')
    return usage_df

#### 2) 전체 유저 이용데이터 분석
* 전체 사용자 리스트 추출
* 사용기간 분석

In [21]:
def analyze_total_user(usage_df):
    user_id_df = usage_df[["user_id"]].drop_duplicates()

    geton_grouped_df = usage_df[["user_id", "geton_datetime"]].groupby(["user_id"])
    getoff_grouped_df = usage_df[["user_id", "getoff_datetime"]].groupby(["user_id"])

    first_use_date_df = geton_grouped_df.min().rename(columns = {"geton_datetime": "first_usage_datetime"})

    geton_last_use_date_df = geton_grouped_df.max().rename(columns = {"geton_datetime": "last_usage_datetime"})
    getoff_last_use_date_df = getoff_grouped_df.max().rename(columns = {"geton_datetime": "last_usage_datetime"})
    last_use_date_df = geton_last_use_date_df

    user_infor_df = pd.merge(user_id_df, first_use_date_df, on = "user_id")
    user_infor_df = pd.merge(user_infor_df, last_use_date_df, on = "user_id")
    user_infor_df["tour_period"] = user_infor_df["last_usage_datetime"] - user_infor_df["first_usage_datetime"]
    user_infor_df["tour_period"]
    
    return user_infor_df

#### 3) 추출1 - 사용기간 고려
관광객으로 유추 가능한 사용자 1차 추출.
1. {user_id, base_date} 쌍을 집합(중복 제거) user_id_and_base_date에 담는다.
    이때, 중복 제거는 {'a1b1c1', '20190101'}, {'a1b1c1', '20190101'} ... 
    => 2회 이상 중복되는 것을 1개만 유지하고 모두 삭제처리한다는 의미.
    * 중복이 생긴 이유는 2019년 1월 1일 a1b1c1이 버스를 3번 탔으면, 3번만큼 튜플이 생기는 것이 당연. 
2. 이후 집합 user_id_and_base_date에 대하여 groupby(user_id).count() 연산을 수행하면,
    각 user가 며칠 버스를 이용하였는지 알 수 있다. (단, 연속적인 며칠 X.) 
    경우1 (1월 1일, 2월 3일, ..., 12월 1일 => 10일): 불연속적인 10일
    경우2 (1월 1일, 2일, 3일, ..., 10일 => 10일): 연속적인 10일
참고로, 2차 추출 단계에서 경우2에 해당하는 튜플만 다시 추출할 것이다.

In [22]:
def extract_user1(usage_df):
    user_id_and_base_date = usage_df[["user_id", "base_date"]].drop_duplicates()
    date_cnt = user_id_and_base_date.groupby(by=["user_id"], as_index=False).count()
    U = list(date_cnt[date_cnt["base_date"] < day]["user_id"])
    return U

#### 4) 추출2 - 사용기간 고려
관광객으로 유추 가능한 사용자 2차 추출.
1. 1차 추출에 의해 식별된 user_id의 인스턴스들을 모아 M2에 저장.
2. 이후 user_id별로 base_date의 max와 min을 추출
3. user_id별로 max - min이 14를 초과하는 경우 삭제 처리
   (연속적으로 며칠 이용했는지 판별 -> 14일 이하인 것만 추출.)

In [23]:
def extract_user2(usage_df, U):
    M2 = usage_df.query('{} in {}'.format("user_id", U))[["user_id", "base_date"]].groupby(by=["user_id"], as_index=False)
    M2_left = M2.max()
    M2_right = M2.min()
    suffixes = ['_max', '_min']

    M2_2 = pd.merge(M2_left, M2_right, on="user_id", how='inner', suffixes=suffixes)
    for postfix in suffixes:
        M2_2["base_date" + postfix] = pd.to_datetime(M2_2["base_date" + postfix], format='%Y%m%d')
    M2_2['diff'] = M2_2["base_date" + suffixes[0]] - M2_2["base_date" + suffixes[1]]

    U2 = list(M2_2[M2_2['diff'] < '%d days' % day]["user_id"].unique()) 
    return U2

#### 5) 추출3 - 방문 정류장 고려
관광객으로 유추 가능한 사용자 3차 추출.
1. 승/하차 정류장 중 '제주국제공항'이 존재하는가?
2. 승/하차 정류장 중 '제주연안여객터미널', '국제여객선터미널'등이 존재하는가?
3. 승/하차 정류장 중 '호텔/여관/펜션' 등이 존재하는가?

In [24]:
def extract_user3(user_df, U2):
    M3 = user_df.query('{} in {}'.format("user_id", U2))

    query_set = []
    for col in [geton_station_name, "getoff_station_name"]:
        for col2 in ['제주국제공항', '제주연안여객터미널', '국제여객선터미널',
                      '호텔', '여관', '펜션', '민박', '관광', '해수욕장', '공원', '파크', '랜드', '휴양림', '박물관', '폭포', '미술관', '오름']:
            query = '{0}.str.contains("{1}") '.format(col, col2)
            query_set.append(query)
    query = ' or '.join(query_set)
    M3 = M3.query(query, engine='python')
    U3 = list(M3["user_id"].unique())
    return U3

### 2. main

In [27]:
start_date = datetime.datetime(2019, 6, 1)
end_date = datetime.datetime(2019, 8, 29)

input_path_list = bus.make_input_path(start_date, end_date)

usage_df = bus.load_tatal_usage_data(input_path_list)
usage_df = data_preprocessing(usage_df)
print('추출 전 user_id의 개수', len(usage_df["user_id"])) 

user_infor_df = analyze_total_user(usage_df)

U = extract_user1(usage_df)
print('1차 추출 결과 user_id의 개수:', len(U))

U2 = extract_user2(usage_df, U)
print('2차 추출 결과 user_id의 개수:', len(U2))

U3 = extract_user3(usage_df, U2)
print('3차 추출 결과 user_id의 개수:', len(U3))
#유의사항: 실제 사용자(관광객) 수는 user_id의 개수보다 크거나 같을 것. (사용자 수는 user_count로 파악할 것.)

user_infor_df["tourist"] = user_infor_df["user_id"].isin(U3).apply(lambda x : int(x))
user_infor_df.to_csv("user_list.csv", encoding="utf-8", index=False)
print("저장완료")

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:51<00:00, 25.93s/it]


추출 전 user_id의 개수 12291789
1차 추출 결과 user_id의 개수: 605906
2차 추출 결과 user_id의 개수: 435662
3차 추출 결과 user_id의 개수: 267735
저장완료


### 3. 결과 출력 

In [26]:
user_infor_df

,user_id,first_usage_datetime,last_usage_datetime,tour_period,tourist
0,f6f372cf8c6732eafc2a82b4f9d7a08bb3b493213ea4ef...,2019-06-01 15:02:25,2019-06-09 20:47:31,8 days 05:45:06,1
1,3f6cfe2429f6e8d62cddc39222993405789b7b9ea2a592...,2019-06-01 15:02:29,2019-06-10 17:12:14,9 days 02:09:45,1
2,107cd874732b48a35df957bf653751154b0234e5573ba5...,2019-06-01 07:20:13,2019-06-07 14:03:34,6 days 06:43:21,0
3,27b16001fafd812b1e5c474cbc60455a9286ad27ecf8e5...,2019-06-01 07:45:59,2019-06-10 07:48:55,9 days 00:02:56,0
4,89c9a1c6e5915df4c192acc54067ee9a743f44eb559f3b...,2019-06-01 17:42:06,2019-06-05 20:24:43,4 days 02:42:37,0
...,...,...,...,...,...
233557,af04d5808a3d8b54e955bd2d12e1e0448b26b1b9916225...,2019-06-10 14:37:04,2019-06-10 14:37:04,0 days 00:00:00,1
233558,8fbe4dff1570744837279dcbd16b96048f31470c59e3fa...,2019-06-10 07:58:38,2019-06-10 07:58:38,0 days 00:00:00,0
233559,e616d1fddb5969605fe902044b963f7531d305a7bd428e...,2019-06-10 16:18:52,2019-06-10 16:18:52,0 days 00:00:00,0
233560,b60d072d772cde597fcc13e22dc21bafa7b9992cc3dc58...,2019-06-10 18:39:06,2019-06-10 18:39:06,0 days 00:00:00,1


# 추출된 임의의 관광객 이동 패턴 출력 예시

In [11]:
usage_df.query('{} == "{}"'.format("user_id", 
                           '250557c8a203138628194b0ed3dfe9c711c3d21d09d984916c4c9546134f22be'))

,user_id,base_date,route_id,route_name,route_no,geton_datetime,geton_station_id,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_id,getoff_station_name,getoff_station_longitude,getoff_station_latitude,user_type,user_count,input_date


In [12]:
usage_df.query('{} == "{}"'.format("user_id", 
                           '055373583405cd9f943d40b199d52570332f3ac91e2f85b726e67da48284d2a8'))

,user_id,base_date,route_id,route_name,route_no,geton_datetime,geton_station_id,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_id,getoff_station_name,getoff_station_longitude,getoff_station_latitude,user_type,user_count,input_date


In [13]:
usage_df.query('{} == "{}"'.format("user_id", 
                           'f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10494887ebb5e470023b')).sort_values(by="geton_datetime")

,user_id,base_date,route_id,route_name,route_no,geton_datetime,geton_station_id,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_id,getoff_station_name,getoff_station_longitude,getoff_station_latitude,user_type,user_count,input_date


In [14]:
usage_df.query('{} == "{}"'.format("user_id", 
                           '1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb8e8822b47397cf3e11')).sort_values(by="geton_datetime")

,user_id,base_date,route_id,route_name,route_no,geton_datetime,geton_station_id,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_id,getoff_station_name,getoff_station_longitude,getoff_station_latitude,user_type,user_count,input_date


In [15]:
usage_df.query('{} == "{}"'.format("user_id", 
                           '68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4fa4afe6255d7b33d58')).sort_values(by="geton_datetime")

,user_id,base_date,route_id,route_name,route_no,geton_datetime,geton_station_id,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_id,getoff_station_name,getoff_station_longitude,getoff_station_latitude,user_type,user_count,input_date


In [16]:
usage_df.query('{} == "{}"'.format("user_id", 
                           'a6583c22116492a72c059e41272ef7aacdc29f6e9138ede6cf5142c26d780fb4'))

,user_id,base_date,route_id,route_name,route_no,geton_datetime,geton_station_id,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_id,getoff_station_name,getoff_station_longitude,getoff_station_latitude,user_type,user_count,input_date
